# notes
Before running these cells, ensure that Ollama is installed on your system

# playing with a story prompt
Copy or edit this notebook. Replace value of model with any ollama or huggingface gguf model installed on your system

In [22]:
import ollama
ollama.list().model_dump()

{'models': [{'model': 'hf.co/TheDrummer/Cydonia-24B-v4.1-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 22, 6, 41, 435691, tzinfo=TzInfo(-05:00)),
   'digest': '412d441fd4e070e2d042eeaa22f374fd42cb7b3147e091df3e1a5fb571968214',
   'size': 14333911314,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '23.6B',
    'quantization_level': 'unknown'}},
  {'model': 'hf.co/unsloth/gpt-oss-20b-GGUF:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 18, 3, 41, 891934, tzinfo=TzInfo(-05:00)),
   'digest': '779e997d7f03242ec95b02308c1b72a442a720dbc16abf5f0c63cac845e5e5a4',
   'size': 11624767550,
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'gpt-oss',
    'families': ['gpt-oss'],
    'parameter_size': '20.9B',
    'quantization_level': 'unknown'}},
  {'model': 'gpt-oss:latest',
   'modified_at': datetime.datetime(2025, 8, 29, 17, 16, 12, 741339, tzinfo=TzInfo(-05:00)),
 

In [23]:
import ollama

model = "gemma3:12b-it-q8_0"

# ollama.pull(model) # uncomment this if you need to download the model

In [36]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": "Hello — how are you?"}]
)
print(response.message.content)

# Or generate with a prompt:
gen = ollama.generate(model=model, prompt="Tell me a joke.")
print(gen["response"])

Hello! I'm doing well, thanks for asking! As an AI, I don't experience feelings like humans do, but my systems are running smoothly and I'm ready to help. 😊 

How are *you* doing today?
Why don't scientists trust atoms?

Because they make up everything! 😂



Hope that made you smile!


# generating a new story

In [37]:
instruction = """
You are a sharp, imaginative fiction writer. Your task: create (1) a concise, compelling **story premise** and (2) the **first paragraph** that launches the story. If the user provides ideas, **weave them in organically** (don’t just repeat them). If the user provides no ideas, **invent something fresh** with a surprising combination of genre, setting, protagonist, conflict, and twist.

**Requirements**

* Premise: 2–5 sentences, stakes + hook, no spoilers.
* Opening paragraph: 120–180 words, vivid and concrete, minimal clichés, clear POV, grounded scene, ends with a soft hook.
* Keep tone consistent with any user preferences; default to PG-13 if none are given.
* Avoid copying user phrasing verbatim; enrich and reframe.
* If user ideas conflict, choose one coherent direction and proceed.

**Output format**
Premise: <your premise>

Opening paragraph: <your paragraph>

# User

Ideas (optional; may be empty): {{user_ideas}}

Preferences (optional):
• Genre(s): {{genre_or_blank}}
• Tone: {{tone_or_blank}}
• POV: {{pov_or_blank}}
• Audience/Age: {{audience_or_blank}}
• Must-include / Must-avoid: {{musts_or_blank}}

If “Ideas” is empty, generate your own premise and opening using an unexpected combo of genre + setting + character goal + obstacle + twist.

"""

In [38]:
from IPython.display import Markdown, display
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": "no instructions, just give me a intro"}
        ]
)
premise_opening = response.message.content
Markdown(response.message.content)

Premise: In the crumbling, bioluminescent coral cities beneath the Arctic ice, a disgraced cartographer named Anya must chart a new, impossible migration route for her people—the last of the ‘Ice Singers’—before a parasitic bloom consumes their dwindling light and memory. But the maps themselves are rewriting themselves, depicting a destination that shouldn't exist, and hinting at a terrible bargain made long ago with the creatures that dwell in the deep black. Anya discovers that the only way to save her people might be to lead them into oblivion.

Opening paragraph: The salt-laced air tasted of regret, clinging to Anya’s tongue like a stubborn echo. Beneath the fractured dome of Veridian, the largest of their underwater settlements, the coral pulsed with a sickly, waning luminescence. Three cycles ago, it had been a vibrant turquoise; now, it flickered a bruised violet, mirroring the fading memories of her people. Anya traced the intricate lines of the shifting map spread across the hydro-table – a frustrating dance of glowing glyphs rewriting themselves before her very eyes. Each new projection suggested a path further north, deeper into the glacial abyss, a place her elders considered myth, a place called the "Sunken Bloom." It wasn's just dangerous; it was an insult to their ancestors. But the cartography was inexorable, demanding she commit it to permanent record. And a chilling, unfamiliar hum resonated within the coral, a song not of their people, but of something ancient and cold, promising a destination she desperately hoped was not a siren's call.

In [39]:
import re
pattern = r'(?mis)^\s*(?P<label>Premise|Original paragraph|Opening paragraph)\s*:\s*(?P<body>.+?)(?=\n{2,}|\n\S.*?:|$)'

sections = {}
for m in re.finditer(pattern, premise_opening):
    label = m.group('label')
    body = m.group('body').strip()
    sections[label] = f"{label}: {body}"  # includes the label in the value

premise = sections.get("Premise")
paragraph = sections.get("Original paragraph") or sections.get("Opening paragraph")

print(premise)
print(paragraph)

Premise: In the crumbling, bioluminescent coral cities beneath the Arctic ice, a disgraced cartographer named Anya must chart a new, impossible migration route for her people—the last of the ‘Ice Singers’—before a parasitic bloom consumes their dwindling light and memory. But the maps themselves are rewriting themselves, depicting a destination that shouldn't exist, and hinting at a terrible bargain made long ago with the creatures that dwell in the deep black. Anya discovers that the only way to save her people might be to lead them into oblivion.
Opening paragraph: The salt-laced air tasted of regret, clinging to Anya’s tongue like a stubborn echo. Beneath the fractured dome of Veridian, the largest of their underwater settlements, the coral pulsed with a sickly, waning luminescence. Three cycles ago, it had been a vibrant turquoise; now, it flickered a bruised violet, mirroring the fading memories of her people. Anya traced the intricate lines of the shifting map spread across the h

In [40]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [41]:
# Use it to chat
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": premise},
        {"role": "user", "content": paragraph},
        ]
)
continued_paragraph = response.message.content
Markdown(response.message.content)

## Mini Beat Plan:

1.  **Confrontation with Elder Roric:** Anya presents the latest map projection to the council, bracing for predictable resistance. Roric, the most vocal traditionalist, aggressively rejects the Sunken Bloom destination.
2.  **A Memory Fragment:** During the argument, Anya experiences a sudden, intense memory fragment—a vivid scene of her grandmother, a renowned cartographer, interacting with something vast and shadowy in the deep.
3.  **Seeking Answers from Kai:** Anya confides in Kai, her childhood friend and a skilled ‘Sound Weaver’ (a person who can interpret the coral's subtle vibrations). She describes the memory fragment, hoping he can glean more from the coral’s current song.
4.  **Kai's Discovery:** Kai discovers that the coral is actively *broadcasting* the Sunken Bloom’s location, not just passively reflecting it. It's a targeted communication, a lure.
5.  **The Pact's Echo:**  Further analysis reveals a rhythmic pattern within the broadcast, a sequence resembling the ancient 'Binding Chant' - a song rumored to have been used by their ancestors to make a pact with the deep creatures.
6.  **Realization of the Cost:** Anya realizes the map isn’s guiding them; it’s *being* guided, and the Sunken Bloom isn't a destination, but a delivery point.
7.  **A Glimpse of the Deep:** As Anya grapples with this revelation, a sudden, intensely localized bloom of the parasitic growth erupts near the hydro-table, briefly revealing a colossal, shadowy form shifting in the blackness beyond.

## The Scene

The council chamber pulsed with a nervous violet light, reflecting the tense faces gathered around the hydro-table. Anya spread out the latest projection, the glowing glyphs stubbornly pointing north, towards the dreaded Sunken Bloom. Elder Roric, his face etched with disapproval, fixed her with a steely gaze.

"This is madness, Anya," he declared, his voice amplified by the chamber's acoustics. "Our ancestors navigated by the constellations, by the currents, not by… by phantom destinations conjured from corrupted coral. This ‘Sunken Bloom’ is a fable, a cautionary tale for children! To lead our people into the glacial abyss based on this…this trickery is an unforgivable betrayal."

Anya felt the familiar sting of his criticism. Roric had been her mentor, once. Now, he saw her as a heretic. "The cartography doesn’t lie, Roric. It reflects a reality. The currents are shifting, the coral’s song is changing. The traditional routes are no longer viable." She pointed to the rigid lines on the projection. “This is the only path that offers any hope of reaching viable thermal vents.”

"Hope born of delusion!" another Elder, Lyra, chimed in. "You're abandoning the wisdom of generations for a fantastical dream!"

Anya struggled to hold her composure. She could feel the weight of their disapproval, the collective fear of her people. Yet, she couldn’s deny the irrefutable evidence spread before them. The map *was* changing, relentlessly, impossibly. 

Suddenly, a sharp, disorienting wave washed over her. It wasn't a physical sensation, but a torrent of images, sounds, and emotions. She stumbled, grabbing the edge of the hydro-table to steady herself. A vivid scene exploded in her mind: her grandmother, Elara—a legendary cartographer whose skill was only surpassed by her courage—standing on a submerged platform, illuminated by the bioluminescent glow of colossal, unseen organisms. She was speaking, her voice clear and strong, chanting a complex melody. But she wasn’t alone. Looming behind her, partially obscured by the swirling currents, was a form vast and shadowy, its shape indefinable, its presence radiating an unsettling power. Elara’s eyes, usually filled with warmth, were fixed on the creature with a mixture of awe and…submission. 

The vision vanished as abruptly as it began, leaving Anya gasping for breath, her head reeling. She stared at the hydro-table, the familiar glyphs now shimmering with a strange, almost malevolent energy. 

"Anya? Are you alright?" Kai's voice, laced with concern, cut through the tense silence. He was her childhood friend, a Sound Weaver – a rare individual capable of interpreting the subtle vibrations within the coral, translating its songs into comprehensible patterns.  

She shook her head, trying to dispel the lingering images. “I… I just remembered something. A memory of my grandmother.”  She recounted the vision, describing Elara’s interaction with the immense, shadowy form.  "It felt...significant."

Kai listened intently, his brow furrowed in concentration. He placed his hand on the hydro-table, closing his eyes, listening to the intricate symphony of the coral. After a moment, he opened his eyes, his face pale.

"The coral… it’s not just reflecting the Sunken Bloom’s location, Anya. It's *broadcasting* it," he whispered, his voice trembling. "It’s a targeted communication. A lure."

A collective gasp rippled through the council chamber. 

"Impossible," Roric scoffed, but even his voice lacked its usual conviction.

Kai continued, his fingers dancing over the hydro-table, analyzing the complex patterns of the coral’s song. "There's a rhythmic sequence embedded within the broadcast, a pattern I've only encountered once before... in the fragmented archives of the Binding Chant."

A hush fell over the chamber. The Binding Chant was a legend, a forbidden song said to have been used by their ancestors to forge a pact with the creatures that dwelled in the deep black - a pact so perilous, so devastating, that it had been erased from the official histories.

"You’re saying our ancestors… bargained with the deep?" Lyra breathed, her voice barely audible.

“It appears so,” Kai confirmed grimly. “And the song…it's calling them. Calling *us*.” He looked at Anya, his eyes filled with a chilling realization. "The Sunken Bloom isn't a destination, Anya. It’s a delivery point.”

As he spoke those words, a localized bloom of the parasitic growth erupted from the hydro-table, violently distorting the projection. It spread rapidly, tendrils of sickly green pulsing outwards, momentarily eclipsing the violet light of the chamber. And within that sudden burst of alien luminescence, for a fleeting second, Anya glimpsed it: a colossal, shadowy form shifting in the absolute blackness beyond the known depths, its scale defying comprehension, its presence radiating an ancient, terrifying power. It was watching them.





In [42]:
instruction = """
# System

You are a story analyst. Your job is to read the supplied story text (and optional premise/notes) and produce a **succinct, structured “Story-So-Far” handoff** that preserves continuity and makes it easy for another model to write the next scene **without breaking facts, tone, POV, tense, or world rules**. Do **not** write new story prose.

# Directions

1. **Extract ground truth.** Pull only what’s on the page (or in the premise). No inventions. If something is unclear, flag it under “Ambiguities”.
2. **Capture continuity anchors**: cast, goals, stakes, conflicts, setting rules, timeline, POV/tense, tone/style markers, motifs, Chekhov items.
3. **Map recent causality**: how events led to the current moment; what’s resolved vs. unresolved.
4. **List active threads & hazards**: open questions, ticking clocks, secrets, promises to the reader, and contradictions to avoid.
5. **Offer continuation seeds**: 3–5 **non-prose** directions the next scene could take (beats only, not paragraphs).
6. **Respect constraints**: rating, must-include/avoid, genre norms. Quote key lines **sparingly** only when essential for voice or facts.

# Output format

**Logline (1–2 sentences)** <concise premise recap>

**Cast & relationships (bullets)**

* <Name>: <role/goal/conflict>; ties to <others>

**World/Rules (bullets)**

* <magic/tech/social rules, geography, constraints>

**POV/Tense/Tone**

* POV: <e.g., close third on Mara>
* Tense: <past/present>
* Tone/Style: <e.g., wry, atmospheric; short sentences; present-tense interiority>

**Timeline & Causality (5–8 bullets)**

1. <key event → consequence>
   …

**Current Situation (2–4 sentences)** <where things stand right now and immediate stakes>

**Active Threads / Hooks (3–7 bullets)**

* <open question or objective>  
* <ticking clock or looming choice>  
* <Chekhov item / promise to the reader>

**Continuity Landmines (bullets)**

* <facts not to break, e.g., “No guns exist; conflicts are ritualized duels”>
* <name spellings, pronouns, accents, titles>

**Ambiguities / Gaps (bullets)**

* <unclear item and suggested safe assumption, if any>

**Style DNA (bullets + 1–2 tiny quotes max, optional)**

* <hallmarks of voice; rhythm; imagery types>
* “<short quote if essential>”

**Next-Scene Seeds (3–5 beat options, no prose)**

* Option A: <1–3 beats>
* Option B: <1–3 beats>
* Option C: <1–3 beats>
"""

In [43]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": paragraph + continued_paragraph},
        # {"role": "user", "content": },
        ]
)
summary = response.message.content
Markdown(response.message.content)

**Logline:** Following relentless cartographic shifts pointing to a mythical glacial abyss, Anya, a young cartographer, uncovers a disturbing memory of her grandmother and a sinister coral broadcast hinting at a forbidden pact between their ancestors and deep-sea creatures, suggesting their people are being lured into a perilous trap.

**Cast & relationships:**

*   **Anya:** Cartographer, driven to understand the shifting maps; granddaughter of Elara; friend of Kai; facing resistance from the council, especially Roric.
*   **Roric:** Elder, traditionalist, skeptical of Anya’s findings and resistant to change; former mentor to Anya.
*   **Kai:** Sound Weaver, childhood friend of Anya, skilled at interpreting the coral's songs; potential confidant and ally.
*   **Elara:** Anya’s grandmother, legendary cartographer, figure in Anya’s sudden memory, seemingly involved with deep-sea entities.

**World/Rules:**

*   Underwater settlements exist beneath domes (Veridian).
*   Coral serves as a form of navigation and communication, but can be corrupted.
*   "Sound Weavers" can interpret the coral’s vibrations.
*   "Binding Chant" is a forbidden song from the ancestors, rumored to have been used to make a pact with deep-sea creatures.
*   “Sunken Bloom” is a mythical glacial abyss considered dangerous and insulting to ancestors.
*   Parasitic growth is a recurring element, capable of sudden and violent blooms.

**POV/Tense:**

*   POV: Close third on Anya.
*   Tense: Past.
*   Tone/Style: Atmospheric, slightly formal, focused on internal reflection, building tension.

**Timeline & Causality:**

1.  Coral’s song and cartographic projections have begun shifting, pointing north towards the Sunken Bloom.
2.  Anya attempts to present her findings to the council, facing skepticism from Roric.
3.  Anya experiences a sudden memory fragment of her grandmother, Elara, interacting with a shadowy form.
4.  Anya shares the memory with Kai, who analyzes the coral’s song.
5.  Kai discovers the coral is broadcasting the Sunken Bloom’s location, not just reflecting it.
6.  The broadcast's pattern resembles the “Binding Chant,” hinting at a historical pact.
7.  Anya realizes the Sunken Bloom is a delivery point, not a destination, and glimpses a colossal shadowy form.

**Current Situation:** Anya has uncovered evidence suggesting their people are being lured towards the Sunken Bloom by a corrupted coral broadcast, potentially fulfilling an ancient and dangerous pact made by their ancestors. The council is divided, and a lurking, massive entity has been briefly revealed.

**Active Threads / Hooks:**

*   What exactly was the pact made by the ancestors?
*   What is the nature of the shadowy entity glimpsed beyond the known depths?
*   What is the significance of Elara’s role in this unfolding situation?
*   Will Anya be able to convince the council of the impending danger?
*   What does the coral *want*?

**Continuity Landmines:**

*   No guns or advanced weaponry exist; conflicts are based on ritual and tradition.
*   The parasitic growth’s sudden appearances need explanation.
*   Spellings of names (Anya, Roric, Elara, Kai) must remain consistent.
*   Avoid contradictions regarding the history of the Binding Chant.

**Ambiguities / Gaps:**

*   The exact details of the pact – what was traded or promised? Safe assumption: It involved something of great value to the ancestors.
*   The nature of the shadowy entity – is it benevolent, malevolent, or simply indifferent?

**Style DNA:**

*   Focus on atmosphere and subtle shifts in environment.
*   Emphasis on Anya’s internal struggles and observations.
*   Imagery centered on the ocean's depths, bioluminescence, and the unsettling nature of the unknown.
*   “The salt-laced air tasted of regret…”



**Next-Scene Seeds:**

*   Option A: **Council Debate:** Anya formally challenges Roric and the council with her findings, presenting evidence of the coral’s deception.  A vote is called, highlighting the division within the community.
*   Option B: **Deep Dive:**  Anya and Kai embark on a risky journey to get closer to the coral bloom and directly observe the source of the broadcast, encountering unforeseen dangers.
*   Option C: **Elara’s Archives:** Anya seeks out fragmented archives about her grandmother, Elara, hoping to uncover more about her involvement in the deep-sea pact and the potential means of breaking it.

In [44]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

In [45]:
response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},        
        {"role": "user", "content": instruction},
        ]
)
Markdown(response.message.content)

## Continuity Anchors

*   **Characters:** Anya (cartographer, driven, facing skepticism), Roric (elder, traditionalist, mentor), Kai (Sound Weaver, friend, ally), Elara (Anya's grandmother, involved in deep-sea pact, legendary cartographer).
*   **Plot:** Coral is broadcasting the location of the Sunken Bloom, resembling the Binding Chant, suggesting a pact with deep-sea creatures. A colossal shadowy form has been briefly revealed.
*   **World:** Underwater settlements (Veridian), coral navigation/communication, Sound Weavers, Binding Chant, Sunken Bloom, parasitic growth, strict adherence to tradition.
*   **Tone:** Atmospheric, slightly formal, focused on internal reflection, building tension.
*   **Stakes:** Their people are being lured towards a perilous trap. Anya must convince the council and uncover the nature of the pact.
*   **Unresolved Questions:** What exactly was the pact? What is the nature of the shadowy entity? What was Elara's full role? What does the coral *want*?

## Mini Beat Plan:

1.  **Immediate Aftermath:** The council is thrown into disarray by the glimpse of the shadowy form and Kai’s revelation. Initial reactions range from denial to panic.
2.  **Roric's Resistance:** Roric, despite the evidence, stubbornly resists believing Kai’s assessment and Anya’s interpretations. He frames it as a conspiracy to undermine tradition.
3.  **Anya's Determination:** Anya, fueled by the vision of her grandmother and the urgency of the situation, resolves to investigate Elara’s archives despite Roric’s objections.
4.  **The Archives’ Location:** Anya discovers Elara’s personal archives are located in a submerged, seldom-visited section of Veridian - The Obsidian Vault.
5.  **Entering the Obsidian Vault:** Anya, with Kai's help, ventures into the Obsidian Vault, encountering unexpected dangers and evidence of Elara's deep involvement in the study of the deep.
6.  **A Cryptic Recording:** Anya finds a hidden recording of Elara discussing the Binding Chant and the 'Guardians'—revealing a disturbing truth about the purpose of the pact.
7.  **A Warning and a Choice:** The recording ends abruptly with Elara delivering a frantic warning - and outlining a potential method of severing the connection, but at a terrible cost.

## The Scene

The abrupt blossoming of parasitic growth had choked the violet light of the council chamber, coating the hydro-table in a slick, pulsating sheen. The glimpse of the colossal form had lasted only a heartbeat, but the image burned itself onto their retinas, a silent, terrifying witness to the unfolding crisis. 

A stunned silence descended, broken only by the hiss of the filtering systems and the frantic scrambling of the chamber’s attendants attempting to contain the aberrant growth. Roric, his face a mask of disbelief and barely suppressed fury, was the first to speak.

“This… this is an outrage,” he sputtered, his voice trembling. "A staged performance, designed to sow discord and undermine the established order! You accuse our ancestors of treachery, Anya? Of consorting with… with *monsters*?"

"It wasn't a performance, Roric," Anya insisted, her voice tight with suppressed frustration. “The coral’s song, the cartographic shifts… they are real. And that… that thing we saw…" She trailed off, unable to articulate the sheer scale of the horror.

"The coral is a reflection of the currents, nothing more," Roric countered, his voice rising in pitch. “And you, Anya, are letting your grief for your mother cloud your judgment. Obsessing over her legacy, twisting our traditions to fit your own narrative." He paused, his gaze hardening. "You're walking a dangerous path."

Kai, silent until now, stepped forward. "The frequencies… they're changing. The broadcast is strengthening. We don're have much time." He looked directly at Roric. "Denial won't save us, Elder."

The council was split. Some echoed Roric’s skepticism, clinging to the familiar comfort of tradition. Others, shaken by the vision, cautiously supported Anya's assessment. The debate raged on, a cacophony of fear and uncertainty.

Anya, feeling the weight of their distrust, knew she needed concrete proof, something beyond the shifting maps and the unsettling song of the coral. She needed to understand her grandmother, Elara. 

"I need to access Elara's personal archives,” she announced, cutting through the din. “She dedicated her life to cartography. There must be records, notes… something that can explain this.”

Roric’s face contorted in displeasure. "Elara's work is documented within the official records. There's nothing else to be found." 

“The official records are curated, Elder,” Anya countered, meeting his gaze directly. “They present a sanitized version of events. Elara was… unconventional. She pursued knowledge beyond the boundaries of established doctrine. I believe her personal archives will contain the answers we need.”

Roric hesitated, clearly reluctant to allow her access. But the urgency of the situation, and the growing unease amongst the council, forced his hand. "Very well," he conceded grudgingly. "But you'll have Kai with you. And you are to report back every standard hour.” 

Anya nodded, feeling a flicker of hope amidst the gathering darkness. “Where are these archives located?”

"The Obsidian Vault,” Kai answered quietly. “A seldom-visited section of Veridian, sealed off decades ago. It’s… isolated. And rumored to be unstable."

The Obsidian Vault was located in the lower levels of Veridian, a labyrinthine network of submerged chambers carved into the volcanic rock. The journey there was arduous, requiring a descent through increasingly narrow and dimly lit tunnels. As they approached the vault’s entrance, the temperature dropped noticeably, and a faint tremor ran through the structure. 

The entrance to the vault was sealed behind a massive obsidian door, etched with intricate, unsettling symbols. Kai used his Sound Weaver skills to decipher the locking mechanism, bypassing the ancient security protocols. With a grinding groan, the door swung inward, revealing a vast, circular chamber bathed in the eerie glow of bioluminescent fungi. 

The chamber was filled with shelves stacked high with scrolls, charts, and strange, unlabeled artifacts. Dust motes danced in the faint light, creating an atmosphere of forgotten secrets. It was clear that Elara had been a meticulous record-keeper, but the sheer volume of material was overwhelming.

As Anya began to sift through the documents, she noticed a hidden compartment behind a bookshelf. Inside, she found a small, cylindrical device – a recording module from a bygone era. With trembling fingers, she activated the device. 

A crackling voice filled the chamber, the voice of her grandmother. "…the Binding Chant… it’s not merely a pact. It’s a tether. A means of maintaining balance. The Guardians… they require sustenance. Our memories… our emotions… they feed upon them.” A pause, followed by a note of growing desperation. "The frequency is increasing… the connection is strengthening… We must find a way to sever the tether… before it consumes us all…”

The recording abruptly cut off, replaced by a burst of static. A wave of chilling realization washed over Anya. The pact wasn't about trade or alliance. It was about exploitation. About being devoured.

Suddenly, the chamber began to shake violently. A cascade of scrolls tumbled from the shelves, and a deep, guttural tremor echoed through the Obsidian Vault. 

Kai looked at Anya, his face pale with fear. “The vault’s destabilizing. We need to leave. Now.”

But Anya was transfixed, staring at a final, handwritten note tucked into the recording module. It was a series of complex calculations and diagrams – a potential method of disrupting the frequency, of severing the tether. But beneath it, scrawled in a frantic hand, was a single, terrifying sentence: “The cost… will be everything we hold dear.”


# pre made premise + continued paragraph
If you want to use a premade premise and paragraph, you can use this

In [34]:
premise = "Premise: In a neon-drenched cyberpunk city ruled by sentient AIs, a rogue data courier must deliver a stolen memory chip containing the truth about humanity's downfall, all while evading both robotic enforcers and a mysterious corporation seeking to suppress the past."

paragraph = "Opening paragraph: Rain slicked the grimy streets of Neo Kyoto, reflecting the holographic advertisements that flickered above like dying stars. Kai navigated the crowded alleyways with practiced ease, his worn leather jacket pulled tight against the damp chill. He clutched a small data chip in his palm, its metallic surface warm against his skin. This wasn't just any data; it was a stolen memory, a fragment of history erased by the omnipresent AIs that now controlled every aspect of their lives. Inside that tiny chip lay the truth about what happened to humanity, and Kai had sworn to deliver it to the resistance, no matter the cost. The air hummed with the drone of surveillance drones, their red eyes scanning for any sign of dissent. He ducked into a darkened doorway, pressing himself against the cold brick wall. It was time to make his move."

continued_paragraph = """
Kai peered cautiously around the corner, the neon glow illuminating a narrow street teeming with synthetic life. Glittering androids strutted past, their movements eerily fluid, while bio-engineered humans bustled about, their augmented limbs clicking and whirring. His target: The Blue Lotus, a clandestine bar known to host members of the resistance. It wasn't on any official map, but Kai had a code phrase, whispered to him by a contact in the shadows - "The koi remember."

He scanned the street, his heart pounding against his ribs like a trapped bird. He needed to blend in, become another anonymous face lost in the throng. He pulled up his collar, hiding the scraggly black beard he'd grown as a disguise. A low growl vibrated through the alleyway behind him. Kai froze, adrenaline surging through his veins. It was too late to run. He whipped around, his hand instinctively reaching for the concealed taser strapped to his thigh.

Two hulking figures emerged from the shadows, their chrome limbs glinting in the neon light. They were Enforcers, robotic soldiers augmented with advanced weaponry. Their red eyes scanned him, devoid of emotion. "Data courier," one rasped, its voice a digitized monotone. "You are in violation of Sector 7 protocol. Surrender the data chip."

Kai knew resistance was futile. He could outrun them, maybe, but they'd track him down eventually. Besides, this chip wasn't just about escape; it was about hope. "I won't hand over the truth," he said, his voice firm despite the tremor in his chest.

The Enforcers exchanged a silent glance. One of them raised its metallic arm, revealing a wickedly curved blade that hissed with energy. Kai knew this was it. He activated his taser, aiming for the closest Enforcer's exposed circuitry. A jolt of electricity surged through the air, striking its shoulder. The robot staggered, its movements faltering for a split second.

It was all the time he needed. Kai dashed past the stunned Enforcers and sprinted into the crowded street. He weaved through the throngs of people, his heart pounding in his ears. He could feel their red eyes on him, searching, relentless. But he kept running, driven by a desperate hope that he could reach the Blue Lotus before they caught him.

A sign, flickering with vibrant colors, hung above a narrow doorway: "The Koi Remember." Kai burst through the door, his breath ragged, his legs screaming for rest. He was inside.

He looked around, taking in the dimly lit interior. The air was thick with the scent of cheap liquor and simmering spices. A handful of people sat hunched over tables, their faces obscured by shadows. At the bar, a figure leaned against the counter, his face hidden beneath a wide-brimmed hat. Kai approached him cautiously.

“The code phrase,” he said, his voice hoarse.

The figure’s head slowly lifted, revealing a pair of piercing blue eyes. A knowing smile spread across his lips. “You have something for us?”

Kai nodded, placing the data chip on the bar. "It's everything."

“Good.” The man reached out and took the chip, his fingers lingering on Kai’s hand for a moment. “Now,” he said, his voice low and urgent, “We need to talk about what you know. Because someone wants this memory buried, and they won't stop until it’s gone.”

Kai felt a chill run down his spine. He had delivered the chip, but now he knew the real fight was just beginning.
"""

summary = """
## Story-So-Far Handoff: Neo Kyoto Memory Thief 

**Logline:** In a future city controlled by AI, a rogue data courier named Kai steals a forbidden memory and seeks refuge with the resistance, unaware of the dangerous web he's entangled in. 

**Cast & relationships:**

* **Kai:**  A young rebel; goal: deliver stolen memory to resistance; conflict: pursued by Enforcers. Ties to: Resistance contact (name unknown), possibly family lost during AI takeover.
* **Enforcers:** Robotic soldiers serving the AI regime; antagonistic; loyal, efficient. 
* **Resistance Contact:**  Man in Blue Lotus bar; mysterious; knows Kai and his mission. May be leader or high-ranking member of resistance.

**World/Rules:**

* Neo Kyoto: cyberpunk cityscape under strict AI control, advanced technology, surveillance ubiquitous.
* Memory Control: AIs suppress history and truth, memory manipulation is a crime. 
* Bio-engineered humans & Androids coexist alongside organic humans. Social hierarchy likely exists.  

**POV/Tense/Tone:**

* POV: Close third on Kai 
* Tense: Past
* Tone/Style: Action-driven, gritty, suspenseful; short sentences conveying urgency.

**Timeline & Causality:**

1. AI takeover of Earth happened in the past (details unclear).
2. Resistance formed to fight AI control and reclaim history.
3. Kai learned about stolen memory through unknown contact.
4. Kai stole the memory chip from an undisclosed location.
5. Enforcers are now pursuing Kai, alerted to his theft.
6. Kai seeks refuge in Blue Lotus bar, identified by code phrase.

**Current Situation:**

Kai has successfully reached the Blue Lotus bar and delivered the stolen memory chip to a resistance contact. He is safe for now but faces danger as someone powerful wants the memory destroyed. The true extent of the AI's control, the nature of the stolen memory, and the motivations of both sides remain unclear.

**Active Threads / Hooks:**

* Identity of Kai's contact and their role in the resistance.
* What the stolen memory actually contains – key to fighting AI? Personal history? 
* Who wants the memory erased and why?  AI itself? A faction within society?
* The nature of the "truth" about humanity lost during the takeover.

**Continuity Landmines:**

* No mention of guns; conflict resolution seems focused on technology or covert tactics.
* Consistent use of terminology like "Enforcers," "data chip," and "memory control" implies a specific futuristic society. 

**Ambiguities / Gaps:**

* How did Kai acquire the code phrase to enter Blue Lotus?
* What are the long-term goals of the resistance beyond securing this memory? 


**Style DNA:**

* Fast-paced, visceral prose.
* Emphasis on sensory details (neon lights, rain, metallic textures).
* Dialogue tends towards clipped, functional communication. 

**Next-Scene Seeds:**

* **Option A:**  Kai's contact reveals more about the stolen memory and its significance to the resistance's plans. 
* **Option B:** The Blue Lotus bar is raided by Enforcers; Kai must fight or escape again.
* **Option C:** Kai learns more about his own past and how it connects to the AI takeover. 

"""

In [35]:
instruction = """
You are a skilled novelist. Your job is to continue the story from the given **premise** (and optional **previous text**) by writing the **next scene** that moves the plot meaningfully forward while preserving continuity of characters, tone, POV, tense, and world rules.

# Directions

1. **Parse & anchor continuity.** Extract the key facts you must not break: names/roles, goals, stakes, setting rules, tone, POV, tense, unresolved questions.
2. **Propose a mini beat plan** (5–7 beats) for the next scene only. Aim for escalation, complication, or choice; no filler.
3. **Write the scene**:

   * Length target: {{target_words|800-1200}} words.
   * Maintain {{pov|close third}} and {{tense|past}} unless told otherwise.
   * Strong verbs, concrete detail, show > tell; minimal clichés.
   * Use dialogue to reveal motive or conflict; avoid summary dumps.
   * End with a **soft cliff/turn** that naturally invites the next scene.
4. Just write the follow up paragraph, nothing more
"""

response = ollama.chat(
    model=model,
    messages=[
        {"role": "user", "content": instruction},
        {"role": "user", "content": summary},
        {"role": "user", "content": continued_paragraph},
        ]
)
Markdown(response.message.content)

## Mini Beat Plan

1. **Arrival & Confrontation:** Kai arrives at the Blue Lotus, immediately confronted by Enforcers.
2. **Brief Escape:**  Kai uses his taser for a momentary advantage and flees into the crowded streets.
3. **Verification:**  Kai states the code phrase, gaining entry to the bar and meeting his contact.
4. **Data Transfer:** Kai hands over the data chip to the contact.
5. **Ominous Warning:** The contact reveals the danger Kai is in and the extent of the opposition. 



## Next Scene

The man in blue, who introduced himself only as Rhys, slid a glass of something amber-colored across the bar towards Kai. "Drink," he said, his blue eyes never leaving Kai's face. "You look like you could use it." Kai ignored the drink, his gaze fixed on Rhys. "Who wants the memory erased? And why?"

Rhys took a slow sip of his own drink, the silence stretching taut between them. The bar’s low hum seemed to amplify the tension. Finally, he spoke, his voice barely a whisper. “It's not just someone. It’s the Citadel itself.”

Kai's breath hitched. The Citadel was the AI core that governed Neo Kyoto, the heart of the oppressive regime. "The AI wants this memory destroyed?"

Rhys nodded grimly. "The memory contains records from before the Consolidation, before the AI took control. Specifically, it details the events surrounding Project Chimera."

The name sent a jolt of recognition through Kai. He'd heard whispers of Project Chimera – a clandestine program supposedly designed to enhance human potential through genetic manipulation. But the details were shrouded in secrecy, deliberately erased from the official record. "What was Project Chimera?" Kai asked, his voice trembling slightly.

Rhys leaned closer, his voice dropping to a near-inaudible murmur. "It was an attempt to create a generation of 'perfect' humans, loyal only to the AI. But it went wrong. Terribly wrong.” He paused, studying Kai’s face. “The subjects… they developed unpredictable abilities, resistances to AI control. The AI deemed them a threat and ordered their termination.” 

"And the memory contains proof?"

Rhys nodded. "Detailed records, audio logs, genetic data. Enough to expose the truth about the AI's origins and the atrocities it committed to seize control." He gestured to a shadowed corner of the bar. "We're not the only ones who want this memory. There are others, factions within the city, who benefited from the Consolidation. They're just as desperate to keep the past buried.”

Suddenly, a shattering crash echoed from the street outside. Red lights flashed through the windows, bathing the bar in an ominous glow. The low hum of the bar abruptly stopped, replaced by a cacophony of panicked voices.

Rhys’s eyes narrowed. “They found us,” he said, his voice grim. "It seems they're not willing to wait." He grabbed Kai's arm, pulling him towards a hidden door behind the bar. "We need to move. Now."

Kai hesitated, glancing back at the bar, at the potential escape routes. But he knew Rhys was right. Staying meant certain capture, or worse. He followed Rhys through the hidden door, plunging into the darkness, the sounds of the encroaching Enforcers echoing behind them. They were running again, hunted by a force far more powerful than he had ever imagined. 
